In [1]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO

In [3]:
def abc(a):
    if type(a) ==str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else :
        z=a
    return z

def kotak_cam(f):
    tables = camelot.read_pdf(f, flavor='stream', pages='all')

    if len(tables) != 0:
        dt = pd.DataFrame(); tmp = pd.DataFrame();
        for i in range(len(tables)):
            tmp = tables[i].df
            if (tmp.shape[1]<5):
                pass
            elif (tmp.shape[1]==5):
                try:
                    idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('deposit').any(), axis=1) ==True].index ][0]
                    tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True)
                except:
                    print("\nAxis-Column headers missing"); pass;
                tmp.columns = ["Date","Narration","Chq/Ref No","Withdrawal(Dr)/Deposit(Cr)","Balance"]
                dt=pd.concat([dt,tmp]).reset_index(drop=True)
            elif (tmp.shape[1]==9):
                try:
                    idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                    tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True)           
                except:
                    try:
                        idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('description').any(), axis=1) ==True].index ][0]
                        tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True)           
                    except:
                        print("\nHtmpC-Column headers missing"); pass
                dt=pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
            else:
                print("\nDifferent structure found"); pass;
    return dt

def kotak_proc01(dt):

    dt = dt[~dt.index.isin(dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c|statement summary|generated on|generated by|computer generated statement|not').any(), axis=1) ==True].index)]
    dt.drop(dt.nunique(dropna=False)[(dt.nunique(dropna=False) == 1)].index, axis=1,inplace=True)             
    dt.iloc[:,-1]=dt.iloc[:,-1].fillna(dt.iloc[:,-2])
    
    try:
        bal=[c for c in dt.columns if "BALANCE" in str(c).upper() ][0]
        dt[bal]=dt[bal].apply( lambda x: abc(x) )
    except: print("\nBalance columns missing")

    dt['flag']=dt.iloc[:,-1].shift(1)
    dt=dt[~dt.index.isin(dt[dt.iloc[:,-1]==dt.iloc[:,-2]].index)]

    try:
        amt=[c for c in dt.columns if "WITHDRAW" in str(c).upper() ][0]
    except:
        try:
            amt=[c for c in dt.columns if "DEPOSIT" in str(c).upper() ][0]
        except: pass   

    dt[[bal,amt]]=dt[[bal,amt]].replace(r'^\s*$', np.nan, regex=True)

    try:
        dt[amt]=dt[amt].apply( lambda x: x.split(' ')[0] if type(x) == str else x )                       
        dt['Wdl1']=dt[amt].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) *-1               
    except Exception as e: 
        print("Error",e)
 

    dt[bal]=dt[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)                                 
    dt['flag']=dt.iloc[:,0].astype(str)+dt['Wdl1'].astype(str) +dt[bal].astype(str)
    dt['flag2']=np.arange(len(dt))

    dt.loc[ dt[['flag']].duplicated(keep=False) , 'flag' ] =dt['flag'] + dt['flag2'].astype(str)  
    dt['flag']=dt['flag'].apply( lambda row : np.nan if 'nannan' in row else row ).fillna(method='ffill')        
    dt['Narration']=dt.groupby('flag')['Narration'].transform( lambda x :''.join(x)) 
    dt['Chq/Ref No']=dt.groupby('flag')['Chq/Ref No'].transform( lambda x :''.join(x))
    dt = dt.drop_duplicates(['flag'], keep='first').iloc[0:,:-3].reset_index(drop=True)

    dt["Debits"] = np.nan; dt["Credits"] = np.nan;
    for j,i in enumerate(dt[amt]):
        if "Cr" in str(i):
            dt["Credits"][j] = dt[amt][j]
        elif "Dr" in str(i):
            dt["Debits"][j] = dt[amt][j]

    dt["Debits"]=dt["Debits"].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)*-1
    dt["Credits"]=dt["Credits"].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)

    try:
        dat=[c for c in dt.columns if "TRANSACTION DATE" in str(c).upper() ][0]
    except:
        try:
            dat=[c for c in dt.columns if "TXN DATE" in str(c).upper() ][0]
        except:
            try:
                dat=[c for c in dt.columns if "DATE" in str(c).upper() ][0]
            except:pass

    try:
        chq=[c for c in dt.columns if "CHQ" in str(c).upper() ][0]
    except:
        try:
            chq=[c for c in dt.columns if "CHEQUE" in str(c).upper() ][0]
        except:pass

    try:
        narr=[c for c in dt.columns if "REMARKS" in str(c).upper() ][0]
    except:
        try:
            narr=[c for c in dt.columns if "PARTICULARS" in str(c).upper() ][0]
        except:
            try:
                narr=[c for c in dt.columns if "DESCRIPTION" in str(c).upper() ][0]
            except:
                try:
                    narr=[c for c in dt.columns if "DETAILS" in str(c).upper() ][0]
                except:
                    try:
                        narr=[c for c in dt.columns if "NARRATION" in str(c).upper() ][0]
                    except:pass

    try:
        deb=[c for c in dt.columns if "DEBIT" in str(c).upper() ][0]
    except:
        try:
            deb=[c for c in dt.columns if "AMOUNT" in str(c).upper() ][0]
        except:
            try:
                deb=[c for c in dt.columns if "WITHDRAW" in str(c).upper() ][0]
            except:pass

    try:
        crd=[c for c in dt.columns if "CREDIT" in str(c).upper() ][0]
    except:
        try:
             crd=[c for c in dt.columns if "DEPOSIT" in str(c).upper() ][0]
        except:pass

    dt = dt[[dat,chq,narr,deb,crd,bal]]
    dt.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]  
    
    dt.drop_duplicates(keep='first', inplace=True)
    
    dt.to_excel("check.xlsx", index=0)
    print("parsed")
    return dt

def kotak_proc02(dt):
    try:        
        if dt.columns[2] == "Date":
            dt.drop('Date', axis=1, inplace=True)
            dt.rename(columns={dt.columns[1] : "Date"}, inplace=True)
    except Exception as e:
        print("Error : ",e)

    dt = dt[~dt.index.isin(dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c|statement summary|generated on|generated by|computer generated statement|not').any(), axis=1) ==True].index)]
    dt.drop(dt.nunique(dropna=False)[(dt.nunique(dropna=False) == 1)].index, axis=1,inplace=True)             
    dt.iloc[:,-1]=dt.iloc[:,-1].fillna(dt.iloc[:,-2])

    try:
        bal=[c for c in dt.columns if "BALANCE" in str(c).upper() ][0]
        dt[bal]=dt[bal].apply( lambda x: abc(x) )
    except: print("\nBalance columns missing")

    try:
        amt=[c for c in dt.columns if "WITHDRAW" in str(c).upper() ][0]
    except:
        try:
            amt=[c for c in dt.columns if "DEPOSIT" in str(c).upper() ][0]
        except:
            try:
                amt=[c for c in dt.columns if "AMOUNT" in str(c).upper() ][0]
            except: pass

    dt[[bal,amt]]=dt[[bal,amt]].replace(r'^\s*$', np.nan, regex=True)

    try:
        dt[amt]=dt[amt].apply( lambda x: x.split(' ')[0] if type(x) == str else x )                       
        dt['Wdl1']=dt[amt].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)             
    except Exception as e: 
        print("Error",e)

    try:
        dt[bal]=dt[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)
    except Exception as e:
        pass
    try:
        dt['flag']=dt['Wdl1'].astype(str) +dt[bal].astype(str)
    except:pass
        
    try:
        dt['flag2']=np.arange(len(dt))
    except:pass

    dt.loc[ dt[['flag']].duplicated(keep=False) , 'flag' ] =dt['flag'] + dt['flag2'].astype(str)  
    dt['flag']=dt['flag'].apply( lambda row : np.nan if 'nannan' in row else row ).fillna(method='ffill')        
    dt['Description']=dt.groupby('flag')['Description'].transform( lambda x :''.join(x)) 
    dt['Chq / Ref number']=dt.groupby('flag')['Chq / Ref number'].transform( lambda x :''.join(x))
    dt['Date']=dt.groupby('flag')['Date'].transform( lambda x :''.join(x))
    dt = dt.drop_duplicates(['flag'], keep='first').iloc[0:,:-3].reset_index(drop=True)

    dt = dt.iloc[:,:-1]

    try:
        drcr=[c for c in dt.columns if "DR" in str(c).upper() ][0]
        amt = [c for c in dt.columns if "AMOUNT" in str(c).upper()][0]
        print(drcr,amt)
        dt["Credits"] = ""
        dt["Debits"] = ""
    except:pass

    try:
        for i in dt.index:
            if dt[drcr][i] == "DR":
                dt["Debits"][i] = dt[amt][i]
            elif dt[drcr][i] == "CR":
                dt["Credits"][i] = dt[amt][i]
            else:
                print(i)
                print("Dr & CR Absent")
    except:pass

    df = dt

    try:
        dat=[c for c in df.columns if "TRANSACTION DATE" in str(c).upper() ][0]
    except:
        try:
            dat=[c for c in df.columns if "TXN DATE" in str(c).upper() ][0]
        except:
            try:
                dat=[c for c in df.columns if "DATE" in str(c).upper() ][0]
            except:pass

    try:
        chq=[c for c in df.columns if "CHQ" in str(c).upper() ][0]
    except:
        try:
            chq=[c for c in df.columns if "CHEQUE" in str(c).upper() ][0]
        except:pass

    try:
        narr=[c for c in df.columns if "REMARKS" in str(c).upper() ][0]
    except:
        try:
            narr=[c for c in df.columns if "PARTICULARS" in str(c).upper() ][0]
        except:
            try:
                narr=[c for c in df.columns if "DESCRIPTION" in str(c).upper() ][0]
            except:
                try:
                    narr=[c for c in df.columns if "DETAILS" in str(c).upper() ][0]
                except:pass

    try:
        bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
        df["bal"]=df[bal].apply( lambda x: abc(x) )
    except: print("\nBalance columns missing")
        
    print(bal)

    try:
        wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
    except:
        try:
            wdl=[c for c in df.columns if "DEBIT" in str(c).upper() ][0]
        except: pass 

    try:
        dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
    except:
        try:
            dep=[c for c in df.columns if "CREDIT" in str(c).upper() ][0]
        except: pass

    print(wdl,dep)
        
    df[[wdl, dep]] = df[[wdl, dep]].replace({"NA":np.nan, "-":np.nan,"0":np.nan,"":np.nan})

    try:
        auto=[c for c in df.columns if "AUTOSWEEP" in str(c).upper() ][0]
        for j,i in enumerate(df[auto]):
            if isnan(i) == False:
                df[wdl][j] = df[auto][j]
            else:
                pass
    except: pass

    try:
        rev=[c for c in df.columns if "REVERSE" in str(c).upper() ][0]
        for j,i in enumerate(df[rev]):
            if isnan(i) == False:
                df[dep][j] = df[rev][j]
            else:
                pass
    except: pass

    try:
        df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
    except:pass
    try:
        df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
    except:pass
    # df.to_csv("check.csv", index=0)
    try:
        df[wdl]=df[wdl].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float) *-1
    except:pass
    try:
        df[dep]=df[dep].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)
    except:pass
    try:
        df[bal]=df[bal].astype(str).apply(lambda x:str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)
    except:pass
    print(df.head())

    try:
        for i in df.index:
            if df["bal"][i] == "Dr":
                df[bal][i] = df[bal][i]*-1
            else:
                pass
    except:pass

    try:
        df.drop(['bal'], axis=1, inplace=True)
    except:pass

    try:
        df.drop([auto], axis=1, inplace=True)
    except:pass

    try:
        df.drop([rev], axis=1, inplace=True)
    except:pass

    df = df[[dat,chq,narr,wdl,dep,bal]]
    df.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]

    df.to_excel("check.xlsx", index=False)

    return df

f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\KOTAK\files\type1\kotak02.pdf'
df = kotak_cam(f)
if len(df.columns) == 5:
    df = kotak_proc01(df)
elif len(df.columns) == 9:
    df = kotak_proc02(df)

Dr / Cr Amount
Balance
Debits Credits
0 Sl. No.        Date                              Description  \
0          04/05/2019     IMPS from VARUN GHEI Ref912416332500   
1          04/05/2019  MB:IMPS to 017101516788Ref 912416823635   
2          07/05/2019              CASH DEPOSIT AT 4611 BYAZAR   
3          07/05/2019  MB:IMPS to 017101516788Ref 912711056460   
4          07/05/2019           MB:SENT TO A TRADING7212860439   

0   Chq / Ref number     Amount Dr / Cr    Balance  Credits   Debits  \
0  IMPS-912416820045  10,000.00      CR  69,300.17  10000.0      NaN   
1  IMPS-912416823590  10,000.00      DR  59,300.17      NaN -10000.0   
2                     34,000.00      CR  93,300.17  34000.0      NaN   
3  IMPS-912711056281  45,000.00      DR  48,300.17      NaN -45000.0   
4    MB-999617566531  46,300.00      DR   2,000.17      NaN -46300.0   

0        bal  
0  69,300.17  
1  59,300.17  
2  93,300.17  
3  48,300.17  
4   2,000.17  


In [4]:
def abc(a):
    if type(a) ==str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else :
        z=a
    return z

def kotak_cam(f):
    tables = camelot.read_pdf(f, flavor='stream', pages='all')

    if len(tables) != 0:
        dt = pd.DataFrame(); tmp = pd.DataFrame();
        for i in range(len(tables)):
            tmp = tables[i].df
            if (tmp.shape[1]<5):
                pass
            elif (tmp.shape[1]==5):
                try:
                    idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('deposit').any(), axis=1) ==True].index ][0]
                    tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True)
                except:
                    print("\nAxis-Column headers missing"); pass;
                tmp.columns = ["Date","Narration","Chq/Ref No","Withdrawal(Dr)/Deposit(Cr)","Balance"]
                dt=pd.concat([dt,tmp]).reset_index(drop=True)
    
            else:
                print("\nDifferent structure found"); pass;
    return dt

def kotak_proc01(dt):

    dt = dt[~dt.index.isin(dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c|statement summary|generated on|generated by|computer generated statement|not').any(), axis=1) ==True].index)]
    dt.drop(dt.nunique(dropna=False)[(dt.nunique(dropna=False) == 1)].index, axis=1,inplace=True)             
    dt.iloc[:,-1]=dt.iloc[:,-1].fillna(dt.iloc[:,-2])
    
    try:
        bal=[c for c in dt.columns if "BALANCE" in str(c).upper() ][0]
        dt[bal]=dt[bal].apply( lambda x: abc(x) )
    except: print("\nBalance columns missing")

    dt['flag']=dt.iloc[:,-1].shift(1)
    dt=dt[~dt.index.isin(dt[dt.iloc[:,-1]==dt.iloc[:,-2]].index)]

    try:
        amt=[c for c in dt.columns if "WITHDRAW" in str(c).upper() ][0]
    except:
        try:
            amt=[c for c in dt.columns if "DEPOSIT" in str(c).upper() ][0]
        except: pass   

    dt[[bal,amt]]=dt[[bal,amt]].replace(r'^\s*$', np.nan, regex=True)

    try:
        dt[amt]=dt[amt].apply( lambda x: x.split(' ')[0] if type(x) == str else x )                       
        dt['Wdl1']=dt[amt].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) *-1               
    except Exception as e: 
        print("Error",e)
 

    dt[bal]=dt[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)                                 
    dt['flag']=dt.iloc[:,0].astype(str)+dt['Wdl1'].astype(str) +dt[bal].astype(str)
    dt['flag2']=np.arange(len(dt))

    dt.loc[ dt[['flag']].duplicated(keep=False) , 'flag' ] =dt['flag'] + dt['flag2'].astype(str)  
    dt['flag']=dt['flag'].apply( lambda row : np.nan if 'nannan' in row else row ).fillna(method='ffill')        
    dt['Narration']=dt.groupby('flag')['Narration'].transform( lambda x :''.join(x)) 
    dt['Chq/Ref No']=dt.groupby('flag')['Chq/Ref No'].transform( lambda x :''.join(x))
    dt = dt.drop_duplicates(['flag'], keep='first').iloc[0:,:-3].reset_index(drop=True)

    dt["Debits"] = np.nan; dt["Credits"] = np.nan;
    for j,i in enumerate(dt[amt]):
        if "Cr" in str(i):
            dt["Credits"][j] = dt[amt][j]
        elif "Dr" in str(i):
            dt["Debits"][j] = dt[amt][j]

    dt["Debits"]=dt["Debits"].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)*-1
    dt["Credits"]=dt["Credits"].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)

    try:
        dat=[c for c in dt.columns if "TRANSACTION DATE" in str(c).upper() ][0]
    except:
        try:
            dat=[c for c in dt.columns if "TXN DATE" in str(c).upper() ][0]
        except:
            try:
                dat=[c for c in dt.columns if "DATE" in str(c).upper() ][0]
            except:pass

    try:
        chq=[c for c in dt.columns if "CHQ" in str(c).upper() ][0]
    except:
        try:
            chq=[c for c in dt.columns if "CHEQUE" in str(c).upper() ][0]
        except:pass

    try:
        narr=[c for c in dt.columns if "REMARKS" in str(c).upper() ][0]
    except:
        try:
            narr=[c for c in dt.columns if "PARTICULARS" in str(c).upper() ][0]
        except:
            try:
                narr=[c for c in dt.columns if "DESCRIPTION" in str(c).upper() ][0]
            except:
                try:
                    narr=[c for c in dt.columns if "DETAILS" in str(c).upper() ][0]
                except:
                    try:
                        narr=[c for c in dt.columns if "NARRATION" in str(c).upper() ][0]
                    except:pass

    try:
        deb=[c for c in dt.columns if "DEBIT" in str(c).upper() ][0]
    except:
        try:
            deb=[c for c in dt.columns if "AMOUNT" in str(c).upper() ][0]
        except:
            try:
                deb=[c for c in dt.columns if "WITHDRAW" in str(c).upper() ][0]
            except:pass

    try:
        crd=[c for c in dt.columns if "CREDIT" in str(c).upper() ][0]
    except:
        try:
             crd=[c for c in dt.columns if "DEPOSIT" in str(c).upper() ][0]
        except:pass

    dt = dt[[dat,chq,narr,deb,crd,bal]]
    dt.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]  
    
    dt.drop_duplicates(keep='first', inplace=True)
    
    dt.to_excel("check.xlsx", index=0)
    print("parsed")
    return dt

f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\KOTAK\files\type2\kotak20.pdf'
df = kotak_cam(f)
if len(df.columns) == 5:
	df = kotak_proc01(df)

parsed


c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_g

In [3]:
def abc(a):
    if type(a) ==str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else :
        z=a
    return z

def kotak_cam(f):
    tables = camelot.read_pdf(f, flavor='stream', pages='all')

    if len(tables) != 0:
        dt = pd.DataFrame(); tmp = pd.DataFrame();
        for i in range(len(tables)):
            tmp = tables[i].df
            if (tmp.shape[1]<5):
                pass
            elif (tmp.shape[1]==5):
                try:
                    idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('deposit').any(), axis=1) ==True].index ][0]
                    tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True)
                except:
                    print("\nAxis-Column headers missing"); pass;
                tmp.columns = ["Date","Narration","Chq/Ref No","Withdrawal(Dr)/Deposit(Cr)","Balance"]
                dt=pd.concat([dt,tmp]).reset_index(drop=True)
            else:
                print("\nDifferent structure found"); pass;

    dt = dt[~dt.index.isin(dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c|statement summary|generated on|generated by|computer generated statement|not').any(), axis=1) ==True].index)]
    dt.drop(dt.nunique(dropna=False)[(dt.nunique(dropna=False) == 1)].index, axis=1,inplace=True)             
    dt.iloc[:,-1]=dt.iloc[:,-1].fillna(dt.iloc[:,-2])
    
    try:
        bal=[c for c in dt.columns if "BALANCE" in str(c).upper() ][0]
        dt[bal]=dt[bal].apply( lambda x: abc(x) )
    except: print("\nBalance columns missing")

    dt['flag']=dt.iloc[:,-1].shift(1)
    dt=dt[~dt.index.isin(dt[dt.iloc[:,-1]==dt.iloc[:,-2]].index)]

    try:
        amt=[c for c in dt.columns if "WITHDRAW" in str(c).upper() ][0]
    except:
        try:
            amt=[c for c in dt.columns if "DEPOSIT" in str(c).upper() ][0]
        except: pass   

    dt[[bal,amt]]=dt[[bal,amt]].replace(r'^\s*$', np.nan, regex=True)

    try:
        dt[amt]=dt[amt].apply( lambda x: x.split(' ')[0] if type(x) == str else x )                       
        dt['Wdl1']=dt[amt].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) *-1               
    except Exception as e: 
        print("Error",e)
 

    dt[bal]=dt[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)                                 
    dt['flag']=dt.iloc[:,0].astype(str)+dt['Wdl1'].astype(str) +dt[bal].astype(str)
    dt['flag2']=np.arange(len(dt))

    dt.loc[ dt[['flag']].duplicated(keep=False) , 'flag' ] =dt['flag'] + dt['flag2'].astype(str)  
    dt['flag']=dt['flag'].apply( lambda row : np.nan if 'nannan' in row else row ).fillna(method='ffill')        
    dt['Narration']=dt.groupby('flag')['Narration'].transform( lambda x :''.join(x)) 
    dt['Chq/Ref No']=dt.groupby('flag')['Chq/Ref No'].transform( lambda x :''.join(x))
    dt = dt.drop_duplicates(['flag'], keep='first').iloc[0:,:-3].reset_index(drop=True)

    dt["Debits"] = np.nan; dt["Credits"] = np.nan;
    for j,i in enumerate(dt[amt]):
        if "Cr" in str(i):
            dt["Credits"][j] = dt[amt][j]
        elif "Dr" in str(i):
            dt["Debits"][j] = dt[amt][j]

    dt["Debits"]=dt["Debits"].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)*-1
    dt["Credits"]=dt["Credits"].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)

    try:
        dat=[c for c in dt.columns if "TRANSACTION DATE" in str(c).upper() ][0]
    except:
        try:
            dat=[c for c in dt.columns if "TXN DATE" in str(c).upper() ][0]
        except:
            try:
                dat=[c for c in dt.columns if "DATE" in str(c).upper() ][0]
            except:pass

    try:
        chq=[c for c in dt.columns if "CHQ" in str(c).upper() ][0]
    except:
        try:
            chq=[c for c in dt.columns if "CHEQUE" in str(c).upper() ][0]
        except:pass

    try:
        narr=[c for c in dt.columns if "REMARKS" in str(c).upper() ][0]
    except:
        try:
            narr=[c for c in dt.columns if "PARTICULARS" in str(c).upper() ][0]
        except:
            try:
                narr=[c for c in dt.columns if "DESCRIPTION" in str(c).upper() ][0]
            except:
                try:
                    narr=[c for c in dt.columns if "DETAILS" in str(c).upper() ][0]
                except:
                    try:
                        narr=[c for c in dt.columns if "NARRATION" in str(c).upper() ][0]
                    except:pass

    try:
        deb=[c for c in dt.columns if "DEBIT" in str(c).upper() ][0]
    except:
        try:
            deb=[c for c in dt.columns if "AMOUNT" in str(c).upper() ][0]
        except:
            try:
                deb=[c for c in dt.columns if "WITHDRAW" in str(c).upper() ][0]
            except:pass

    try:
        crd=[c for c in dt.columns if "CREDIT" in str(c).upper() ][0]
    except:
        try:
             crd=[c for c in dt.columns if "DEPOSIT" in str(c).upper() ][0]
        except:pass

    dt = dt[[dat,chq,narr,deb,crd,bal]]
    dt.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]  
    
    dt.drop_duplicates(keep='first', inplace=True)
    
    dt.to_excel("check.xlsx", index=0)
    print("parsed")
    return dt

f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\KOTAK\files\type2\kotak20.pdf'
df = kotak_cam(f)

parsed


c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_g